## 06. PyTorch Experiment Tracking

- https://github.com/mrdbourke/pytorch-deep-learning/blob/main/07_pytorch_experiment_tracking.ipynb

Different ways to track machine learning experiments

![Alt text](image-3.png)

![Alt text](image-4.png)

### 0. Getting setup

In [1]:
# try:
#     import torch
#     import torchvision
#     assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
#     assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")
# except:
#     print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
#     !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#     import torch
#     import torchvision
#     print(f"torch version: {torch.__version__}")
#     print(f"torchvision version: {torchvision.__version__}")

In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

In [3]:
import sys
import os

In [4]:
os.getcwd()

'/Users/sguys99/Desktop/project/pytorch-study/zero-to-mastery'

In [5]:
from going_modular import data_setup, engine

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

### 1. Get data

In [8]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str, 
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.
    
    Returns:
        pathlib.Path to downloaded data.
    
    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)
        
        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...") 
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)
    
    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] data/pizza_steak_sushi directory exists, skipping download.


PosixPath('data/pizza_steak_sushi')

### 2. Create Datasets and DataLoaders

앞에서 만든 create_dataloaders() 함수를 사용할 것이다.   
여기서  torchvision.models를 사용하여 전이학습을 진행할 예정이기 때문에 모델에 맡는 변환 작업을 진행해야한다.   

이미지 텐서를 변환하는 방법은 두가지가 있다.  
- Manually created transforms using torchvision.transforms.
- Automatically created transforms using torchvision.models.MODEL_NAME.MODEL_WEIGHTS.DEFAULT.transforms().
    - Where MODEL_NAME is a specific torchvision.models architecture, MODEL_WEIGHTS is a specific set of pretrained weights and DEFAULT means the "best available weights".

위 방법은 06 챕터에서 실시한 것이다.  

먼저 torchvision.transforms 파이프라인을 수동으로 생성하는 예제를 살펴보겠습니다(이 방법으로 생성하면 다양한 사용자 정의가 가능하지만 변환 결과가 전이학습 모델과 일치하지 않으면 잠재적으로 성능이 저하될 수 있습니다).

우리가 확인해야 할 주요 수동 변환은 모든 이미지가 ImageNet 형식으로 정규화되어 있는지 여부입니다(이는 사전 학습된 torchvision.models가 모두 ImageNet에서 사전 학습되었기 때문입니다).

예를 들어 다음과 같이 말이다. 

```python
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
```

#### 2.1 Create DataLoaders using manually created transforms

In [9]:
# Setup directories
train_dir = image_path / "train"
test_dir = image_path / "test"

# Setup ImageNet normalization levels (turns all images into similar distribution as ImageNet)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Create transform pipeline manually
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])           
print(f"Manually created transforms: {manual_transforms}")

# Create data loaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms, # use manually created transforms
    batch_size=32
)

train_dataloader, test_dataloader, class_names

Manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


(<torch.utils.data.dataloader.DataLoader at 0x103b75160>,
 ['pizza', 'steak', 'sushi'])

#### 2.2 Create DataLoaders using automatically created transforms

In [10]:
# Setup dirs
train_dir = image_path / "train"
test_dir = image_path / "test"

# Setup pretrained weights (plenty of these available in torchvision.models)
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# Get transforms from weights (these are the transforms that were used to obtain the weights)
automatic_transforms = weights.transforms() 
print(f"Automatically created transforms: {automatic_transforms}")

# Create data loaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=automatic_transforms, # use automatic created transforms
    batch_size=32
)

train_dataloader, test_dataloader, class_names

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x103cd6fd0>,
 ['pizza', 'steak', 'sushi'])

### 3. Getting a pretrained model, freezing the base layers and changing the classifier head

여러 모델링 실험을 실행하고 추적하기 전에 하나의 실험을 실행하고 추적하는 것이 어떤지 살펴봅시다.

데이터가 준비되었으므로 다음으로 필요한 것은 모델입니다.

torchvision.models.efficientnet_b0() 모델을 다운로드하여, 우리 데이터로 사용할 수 있도록  준비해보자.

In [11]:
# Note: This is how a pretrained model would be created in torchvision > 0.13, it will be deprecated in future versions.
# model = torchvision.models.efficientnet_b0(pretrained=True).to(device) # OLD 

# Download the pretrained weights for EfficientNet_B0
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # NEW in torchvision 0.13, "DEFAULT" means "best weights available"

# Setup the model with the pretrained weights and send it to the target device
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

# View the output of the model
# model

In [12]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

이제 사전 학습된 모델을 특징 extractor 모델로 전환해 보겠습니다.  
기본적으로 모델의 기본 레이어를 고정하고(입력 이미지에서 특징을 추출하는 데 사용할 것입니다) 작업 중인 클래스의 수에 맞게 분류기 헤드(출력 레이어)를 변경합니다(피자, 스테이크, 초밥의 세 가지 클래스가 있습니다).

참고: 특징 추출기 모델을 만드는 아이디어(여기서 하는 일)는 06편에서 더 자세히 다루었습니다.

In [13]:
for param in model.features. parameters():
    param.requires_grad = False
    
set_seeds()

model.classifier = torch.nn.Sequential(
    nn.Dropout(p = 0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features = len(class_names),
              bias = True).to(device)) 

In [14]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [15]:
from torchinfo import summary

# Get a summary of the model (uncomment for full output)
summary(model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape" (batch_size, color_channels, height, width)
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

/Users/sguys99/opt/anaconda3/envs/torch/lib/python3.9/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

![Alt text](image-5.png)

### 4. Train model and track results

이제 손실함수와 옵티마이저를 준비하자.

In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

#### Adjust train() function to track results with SummaryWriter()¶

이전에는 모델마다 하나씩 여러 개의 파이썬 딕셔너리를 사용하여 모델링 실험을 추적했습니다.

하지만 실험을 몇 개 이상 실행하는 경우 이 작업이 복잡해질 수 있다는 것을 상상할 수 있습니다.

걱정하지 마세요. 더 나은 옵션이 있으니까요!   

PyTorch의 torch.utils.tensorboard.SummaryWriter() 클래스를 사용하여 모델 학습 진행 상황의 다양한 부분을 파일에 저장할 수 있습니다.
기본적으로 SummaryWriter() 클래스는 모델에 대한 다양한 정보를 log_dir 매개변수로 설정된 파일에 저장합니다.
log_dir의 기본 위치는 runs/CURRENT_DATETIME_HOSTNAME 아래에 있으며, 여기서 호스트 이름은 사용자 컴퓨터 이름입니다.
물론 실험이 추적되는 위치를 변경할 수 있습니다(파일 이름은 원하는 대로 사용자 지정 가능).

SummaryWriter()의 출력은 TensorBoard 형식으로 저장됩니다.
TensorBoard는 TensorFlow 딥 러닝 라이브러리의 일부로, 모델의 다양한 부분을 시각화하는 데 탁월한 방법입니다.
모델링 실험 추적을 시작하기 위해 기본 SummaryWriter() 인스턴스를 만들어 보겠습니다.

In [18]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

이제 writer를 사용하려면 새로운 훈련 루프를 작성하거나 05에서 만든 기존 train() 함수를 조정할 수 있습니다.  
후자를 선택하겠습니다.  

engine.py에서 train() 함수를 가져와서 writer를 사용하도록 조정하겠습니다.   
구체적으로, train() 함수에 모델의 훈련 및 테스트 손실과 정확도 값을 기록하는 기능을 추가하겠습니다.  
writer.add_scalars(main_tag, tag_scalar_dict)를 사용하여 이 작업을 수행할 수 있습니다:  

- main_tag(string): 추척할 스칼라의 이름(ex: 'Accuracy')
- tag_scalar_dict(dict): 추적할 딕셔너리 값(ex: {"train_loss": 0.3454})
* 손실 및 정확도 값은 일반적으로 스칼라(단일 값)이기 때문에 이 메서드를 add_scalars()라고 합니다.

값 추적이 끝나면 writer.close()를 호출하여 추적할 값 찾기를 중단하라고 지시합니다.

train() 수정을 시작하기 위해 engine.py에서 train_step() 및 test_step()도 임포트합니다.

참고: 코드의 거의 모든 곳에서 모델에 대한 정보를 추적할 수 있습니다. 하지만 모델이 훈련하는 동안(훈련/테스트 루프 내에서) 실험을 추적하는 경우가 많습니다.

또한, torch.utils.tensorboard.SummaryWriter() 클래스에는 모델의 계산 그래프를 추적하는 add_graph()와 같이 모델/데이터에 대한 다양한 정보를 추적하는 여러 가지 메서드가 있습니다. 더 많은 옵션은 SummaryWriter() 설명서를 참조하세요.   

In [19]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.engine import train_step, test_step

In [20]:
def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device) -> Dict[str, List]:
    
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model = model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device
                                           )
        
        test_loss, test_acc = test_step(model = model,
                                        dataloader=test_dataloader,
                                        loss_fn = loss_fn,
                                        device=device
                                        )
        
        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        
        ### New: Experiment tracking ###
        # Add loss results to SummaryWriter
        writer.add_scalars(main_tag="Loss", 
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        # Add accuracy results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy", 
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc}, 
                           global_step=epoch)
        
        # Track the PyTorch model architecture
        writer.add_graph(model=model, 
                         # Pass in an example input
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))
    
    writer.close()
    
    return results

5 epochs 만 시도해보자.

In [21]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0901 | train_acc: 0.4102 | test_loss: 0.8925 | test_acc: 0.6714
Epoch: 2 | train_loss: 0.8940 | train_acc: 0.6562 | test_loss: 0.8082 | test_acc: 0.7746
Epoch: 3 | train_loss: 0.7455 | train_acc: 0.8398 | test_loss: 0.7417 | test_acc: 0.7642
Epoch: 4 | train_loss: 0.7817 | train_acc: 0.6953 | test_loss: 0.6820 | test_acc: 0.8144
Epoch: 5 | train_loss: 0.6320 | train_acc: 0.7734 | test_loss: 0.6400 | test_acc: 0.8665
